# IFC OpenShell Voice Assistant - Prototype #5


In [ ]:
%%html
<iframe src="https://ifcjs.github.io/web-ifc-viewer/example/index" width="500" height="300"></iframe>

In [ ]:
#required definitions
import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re
import pyttsx3
import webbrowser

sensitive_keywords = ["colour", "dimension", "element", "file"]
colours = ["red","blue","yellow"]
dimensions = ["x","y","height"]
elements = ["wall","window","door"]
files = ["4","5","6"]


def text_to_voice (text):

    engine = pyttsx3.init();
    engine.say(text);
    # engine.setProperty('voice', voices[2].id)
    engine.setProperty("rate", 155)
    engine.runAndWait() ;


def get_voice():

    raw_text = {}
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)
        r.pause_threshold = 0.8
        r.dynamic_energy_threshold = True
        audio = r.listen(source)
        raw_text = r.recognize_google(audio)
    return raw_text


def nlp(raw_text, sensitive_keywords):

    command_type = [x for x in sensitive_keywords if x in raw_text]
    if command_type[0] == 'colour':
        command_sup = [x for x in colours if x in raw_text]
    elif command_type[0] == 'dimension':
        command_sup = [x for x in dimensions if x in raw_text]
    elif command_type[0] == 'element':
        command_sup = [x for x in elements if x in raw_text]
    elif command_type[0] == 'file':
        command_sup = [x for x in files if x in raw_text]
    return command_type, command_sup


def open_file():

    text_to_voice (f' Can you please give me the file name')
    get_voice_text = get_voice()
    file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]

    if [x for x in file_name_ if x in files]:
        
        text_to_voice (f'Ok, I will open the file {file_name_}')
        webbrowser.open(f'{file_name_}.txt')

    else:
        
        while bool([x for x in file_name_ if x in files])==False:
            
            text_to_voice (f' Sorry, but I can only open file {files}, Can you please give me the file name?')
            get_voice_text = get_voice()
            file_name_ = re.findall('\d*\.?\d+',get_voice_text)[0]
            if [x for x in file_name_ if x in files]:
                text_to_voice (f'Ok, I will open the file {file_name_}')
                webbrowser.open(f'{file_name_}.txt')

def change_wall_color():
    
    text_to_voice (f' Can you please give me the desired colour')
    get_voice_text = get_voice()
    colour_name_ = [x for x in get_voice_text if x in colours]

    if [x for x in colours if x in get_voice_text]:
        
        text_to_voice (f'Ok, I will change the colour to {colour_name_}')
        """a command to change the colour of the wall"""
        
    else:
        
        while bool([x for x in colours if x in get_voice_text])==False:
            
            text_to_voice (f' Sorry, but I can only change to colour to {colours},Can you please give me the desired colour?')
            get_voice_text = get_voice()
            colour_name_ = [x for x in colours if x in get_voice_text]
            
            if [x for x in colour_name_ if x in colours]:
                
                text_to_voice (f'Ok, I will change the colour to {colour_name_}')
                """a command to change the colour of the wall"""


             
def change_dimension():
    pass


def create_element():
    pass


def get_action(command_type, command_sup):

    if command_type[0] == 'colour':
        change_wall_color()
        
    elif command_type[0] == 'dimension':
        change_dimension()
        
    elif command_type[0] == 'element':
        create_element()
        
    elif command_type[0] == 'file':
        open_file()           

In [ ]:
##Create Wall
import uuid
import time
import tempfile

import ifcopenshell
import numpy
import apiai
import speech_recognition as sr
import re

##IFC Definitions and imports
#Variables directions
O = 0., 0., 0.
X = 1., 0., 0.
Y = 0., 1., 0.
Z = 0., 0., 1.


#open a file/Import this from mail file somehow!!!!
ifcfile = ifcopenshell.open("./two_wall.ifc")

# Obtain references to instances defined in template !!! Define the filename with code
owner_history = ifcfile.by_type("IfcOwnerHistory")[0]
project = ifcfile.by_type("IfcProject")[0]
context = ifcfile.by_type("IfcGeometricRepresentationContext")[0]
filename = "two_wall.ifc"


# Helper function definitions

# Creates an IfcAxis2Placement3D from Location, Axis and RefDirection specified as Python tuples
def create_ifcaxis2placement(ifcfile, point=O, dir1=Z, dir2=X):
    point = ifcfile.createIfcCartesianPoint(point)
    dir1 = ifcfile.createIfcDirection(dir1)
    dir2 = ifcfile.createIfcDirection(dir2)
    axis2placement = ifcfile.createIfcAxis2Placement3D(point, dir1, dir2)
    return axis2placement

# Creates an IfcLocalPlacement from Location, Axis and RefDirection, specified as Python tuples, and relative placement
def create_ifclocalplacement(ifcfile, point=O, dir1=Z, dir2=X, relative_to=None):
    axis2placement = create_ifcaxis2placement(ifcfile,point,dir1,dir2)
    ifclocalplacement2 = ifcfile.createIfcLocalPlacement(relative_to,axis2placement)
    return ifclocalplacement2

# Creates an IfcPolyLine from a list of points, specified as Python tuples
def create_ifcpolyline(ifcfile, point_list):
    ifcpts = []
    for point in point_list:
        point = ifcfile.createIfcCartesianPoint(point)
        ifcpts.append(point)
    polyline = ifcfile.createIfcPolyLine(ifcpts)
    return polyline
    
# Creates an IfcExtrudedAreaSolid from a list of points, specified as Python tuples
def create_ifcextrudedareasolid(ifcfile, point_list, ifcaxis2placement, extrude_dir, extrusion):
    polyline = create_ifcpolyline(ifcfile, point_list)
    ifcclosedprofile = ifcfile.createIfcArbitraryClosedProfileDef("AREA", None, polyline)
    ifcdir = ifcfile.createIfcDirection(extrude_dir)
    ifcextrudedareasolid = ifcfile.createIfcExtrudedAreaSolid(ifcclosedprofile, ifcaxis2placement, ifcdir, extrusion)
    return ifcextrudedareasolid
    
create_guid = lambda: ifcopenshell.guid.compress(uuid.uuid1().hex)

# IFC hierarchy creation
site_placement = create_ifclocalplacement(ifcfile)
site = ifcfile.createIfcSite(create_guid(), owner_history, "Site", None, None, site_placement, None, None, "ELEMENT", None, None, None, None, None)

building_placement = create_ifclocalplacement(ifcfile, relative_to=site_placement)
building = ifcfile.createIfcBuilding(create_guid(), owner_history, 'Building', None, None, building_placement, None, None, "ELEMENT", None, None, None)

storey_placement = create_ifclocalplacement(ifcfile, relative_to=building_placement)
elevation = 0.0
building_storey = ifcfile.createIfcBuildingStorey(create_guid(), owner_history, 'Storey', None, None, storey_placement, None, None, "ELEMENT", elevation)

container_storey = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Building Container", None, building, [building_storey])
container_site = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Site Container", None, site, [building])
container_project = ifcfile.createIfcRelAggregates(create_guid(), owner_history, "Project Container", None, project, [site])

## Write the template to a temporary file 
# temp_handle, temp_filename = tempfile.mkstemp(suffix=".ifc")
# with open(temp_filename, "w+") as f:
#     f.write(template)
 


##Voice Recognition Defs and Imports
# Voice to text
def get_voice():
    r = sr.Recognizer()
    mic = sr.Microphone()
    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)
        r.pause_threshold = 0.8
        r.dynamic_energy_threshold = True
        audio = r.listen(source)
        
    raw_text = r.recognize_google(audio)
    return raw_text

import pyttsx3;
def tts (text):
    engine = pyttsx3.init();
    engine.say(text);
    # engine.setProperty('voice', voices[2].id)
    engine.setProperty("rate", 155)
    engine.runAndWait() ;



#Wall Input Conversations
def wall_inputs():
    text = []
    tts (f"Please enter X dimension\n")
    text = get_voice()
    x_dimension = float(re.findall('\d*\.?\d+', text)[0])
    tts (f" X dimension is {x_dimension}")
    
    

    text = []
    tts (f"Please enter y dimension\n")
    text = get_voice()
    y_dimension = float(re.findall('\d*\.?\d+', text)[0])
    #print(y_dimension)
    tts (f" Y dimension is {y_dimension}")
    

    text = []
    tts (f"Please enter the height\n")
    text = get_voice()
    z_dimension = float(re.findall('\d*\.?\d+', text)[0])
    #print(z_dimension)
    tts (f" Height is {z_dimension}")
    

    return x_dimension, y_dimension, z_dimension

####
x_dimension, y_dimension, z_dimension = wall_inputs()

#center cordinates

def create_origin():
    text = []
    tts (f"Please give me the x and y values of the closest point to origin\n")
    text = get_voice()
    origin_x = float(re.findall('\d*\.?\d+', text)[0])
    origin_y = float(re.findall('\d*\.?\d+', text)[1])
    tts (f"Closest point is {origin_x} comma {origin_y}")
    return origin_x, origin_y


origin_x, origin_y = create_origin()

#coordinate creation
point_list_extrusion_area = [(origin_x, origin_y, 0.0), (x_dimension + origin_x, origin_y, 0.0), (x_dimension + origin_x, y_dimension + origin_y, 0.0), (origin_x, y_dimension+ origin_y, 0.0), (origin_x, origin_y, 0.0)]



#Create Wall
wall_placement = create_ifclocalplacement(ifcfile, relative_to=storey_placement)
polyline = create_ifcpolyline(ifcfile, [(0.0, 0.0, 0.0), (5.0, 0.0, 0.0)])
axis_representation = ifcfile.createIfcShapeRepresentation(context, "Axis", "Curve2D", [polyline])

extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
solid = create_ifcextrudedareasolid(ifcfile, point_list_extrusion_area, extrusion_placement, (0.0, 0.0, 1.0), z_dimension)
body_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [solid])

product_shape = ifcfile.createIfcProductDefinitionShape(None, None, [axis_representation, body_representation])

wall = ifcfile.createIfcWallStandardCase(create_guid(), owner_history, "Wall", "An awesome wall", None, wall_placement, product_shape, None)

# Write the contents of the file to disk
ifcfile.write(filename)

In [ ]:
#Length of wall
#Create wall placement to refer
def refer_a_wall():
    walls = ifcfile.by_type('IfcWall')
    print(*enumerate(walls), sep='\n')
refer_a_wall()
tts (f"Please select a wall from list\n")
text = []
text = get_voice()
wall_number = int(e.findall('\d*\.?\d+', text)[0])

##Cordinates of a wall
walls = ifcfile.by_type('IfcWall')
x = walls[wall_number].Representation.Representations[1].Items[0].SweptArea.OuterCurve.Points
wall_origin_x = x[0][0][0]
wall_origin_y = x[0][0][1]
x_index1 = x[1][0][0]
x_index2 = x[0][0][0]
y_index1 = x[2][0][1]
y_index2 = x[1][0][1]

height_of_wall = walls[wall_number].Representation.Representations[1].Items[0][3]

if x_index2 - x_index1 > 0:
    wall_x_length = round(x_index2 - x_index1, 3)
else :
    wall_x_length = round((-1)*(x_index2 - x_index1), 3)

if y_index2 - y_index1 > 0:
    wall_y_length = round(y_index2 - y_index1, 3)
else :
    wall_y_length = round((-1)*(y_index2 - y_index1), 3)

if wall_x_length > wall_y_length :
    length_of_wall = wall_x_length
    width_of_wall = wall_y_length
elif wall_x_length < wall_y_length :
    length_of_wall = wall_y_length
    width_of_wall = wall_x_length

tts (f"Length of the wall is {length_of_wall}\n")

In [ ]:
#Wall Heigth
walls = ifcfile.by_type("IfcWall")
def refer_a_wall():
    walls = ifcfile.by_type('IfcWall')
    print(*enumerate(walls), sep='\n')
refer_a_wall()
tts (f"Select the wall\n")
text = []
text = get_voice()
wall_number = int(re.findall('\d*\.?\d+', text)[0])
wall_height = walls[2].Representation.Representations[1].Items[0][3]
tts (f"Ok wall height is {wall_height} \n")


In [ ]:
#Number of element
answers = ["yes", "no"]
element_type = "window"
if element_type == "wall":
    number_of_elements = len(ifcfile.by_type("IfcWall"))
elif element_type == "window":
    number_of_elements = len(ifcfile.by_type("IfcWindow"))
elif element_type == "door":
    number_of_elements = len(ifcfile.by_type("IfcDoor"))
tts (f"You have {number_of_elements} {element_type} in your model. Would you like to hear the id\n")
raw_text = get_voice()

if ([x for x in answers if x in raw_text][0]) == 'yes':

    tts (f"The IDs are {short_wall_index} ")

elif ([x for x in answers if x in raw_text][0]) == 'no':

    tts (f"ok ")

In [ ]:
##Window coordinate calculation 
#Create wall placement to refer
def refer_a_wall():
    walls = ifcfile.by_type('IfcWall')
    print(*enumerate(walls), sep='\n')
refer_a_wall()
tts (f"Please select a wall from list\n")
text = []
text = get_voice()
wall_number = int(re.findall('\d*\.?\d+', text)[0])
wall = ifcfile.by_type("IfcWall")[wall_number]
wall_placement = wall[5]

#Window and related opening measurements
def locate_window():
    text = []
    tts (f"Please locate the window on wall, distance from origin and height from ground\n")
    text = get_voice()
    window_distance = float(re.findall('\d*\.?\d+', text)[0])
    height_from_ground = float(re.findall('\d*\.?\d+', text)[1])
    tts (f"Distance from origin is {window_distance} and height is {height_from_ground}")
    return window_distance, height_from_ground

window_distance, height_from_ground = locate_window()

tts (f"Please define length and height of the window \n")
text = []
text = get_voice()
window_length = float(re.findall('\d*\.?\d+', text)[0])
window_height = float(re.findall('\d*\.?\d+', text)[1])


##Cordinates of a wall
x = walls[wall_number].Representation.Representations[1].Items[0].SweptArea.OuterCurve.Points
wall_origin_x = x[0][0][0]
wall_origin_y = x[0][0][1]
x_index1 = x[1][0][0]
x_index2 = x[0][0][0]
y_index1 = x[2][0][1]
y_index2 = x[1][0][1]

height_of_wall = walls[wall_number].Representation.Representations[1].Items[0][3]

if x_index2 - x_index1 > 0:
    wall_x_length = round(x_index2 - x_index1, 3)
else :
    wall_x_length = round((-1)*(x_index2 - x_index1), 3)

if y_index2 - y_index1 > 0:
    wall_y_length = round(y_index2 - y_index1, 3)
else :
    wall_y_length = round((-1)*(y_index2 - y_index1), 3)

if wall_x_length > wall_y_length :
    length_of_wall = wall_x_length
    width_of_wall = wall_y_length
    point_list_opening_extrusion_area = [(x[0][0][0], x[0][0][1], 0.0), (x[0][0][0]+window_length, x[0][0][1], 0.0), (x[0][0][0]+window_length, x[2][0][1], 0.0), (x[0][0][0], x[2][0][1], 0.0), (x[0][0][0], x[0][0][1], 0.0)]
    point_list_window_extrusion_area = [(x[0][0][0], ((x[2][0][1]+x[1][0][1])/2)-0.01, 0.0), (x[0][0][0]+window_length, ((x[2][0][1]+x[1][0][1])/2)-0.01, 0.0), (x[0][0][0]+window_length, ((x[2][0][1]+x[1][0][1])/2)+0.01, 0.0), (x[0][0][0], ((x[2][0][1]+x[1][0][1])/2)+0.01, 0.0), (x[0][0][0], ((x[2][0][1]+x[1][0][1])/2)-0.01, 0.0)]
elif wall_x_length < wall_y_length :
    length_of_wall = wall_y_length
    width_of_wall = wall_x_length
    point_list_opening_extrusion_area = [(x[0][0][0], x[0][0][1], 0.0), (x[1][0][0], x[0][0][1], 0.0), (x[1][0][0], x[0][0][1]+window_length, 0.0), (x[0][0][0], x[0][0][1]+window_length, 0.0), (x[0][0][0], x[0][0][1], 0.0)]
    point_list_window_extrusion_area = [(((x[0][0][0]+x[1][0][0])/2)-0.01, x[0][0][1], 0.0), (((x[0][0][0]+x[1][0][0])/2)+0.01, x[0][0][1], 0.0), (((x[0][0][0]+x[1][0][0])/2)+0.01, x[0][0][1]+window_length, 0.0), (((x[0][0][0]+x[1][0][0])/2)-0.01, x[0][0][1]+window_length, 0.0), (((x[0][0][0]+x[1][0][0])/2)-0.01, x[0][0][1], 0.0)]


    

In [ ]:
# Create and associate an opening for the window in the wall
opening_placement = create_ifclocalplacement(ifcfile, (window_distance, 0.0, height_from_ground), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), wall_placement)
opening_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
#point_list_opening_extrusion_area = [(0.0, -0.1, 0.0), (3.0, -0.1, 0.0), (3.0, 0.1, 0.0), (0.0, 0.1, 0.0), (0.0, -0.1, 0.0)]
opening_solid = create_ifcextrudedareasolid(ifcfile, point_list_opening_extrusion_area, opening_extrusion_placement, (0.0, 0.0, 1.0), window_height)
opening_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [opening_solid])
opening_shape = ifcfile.createIfcProductDefinitionShape(None, None, [opening_representation])
opening_element = ifcfile.createIfcOpeningElement(create_guid(), owner_history, "Opening", "An awesome opening", None, opening_placement, opening_shape, None)
ifcfile.createIfcRelVoidsElement(create_guid(), owner_history, None, None, wall, opening_element)

# Create a simplified representation for the Window
window_placement = create_ifclocalplacement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), opening_placement)
window_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
#point_list_window_extrusion_area = [(0.0, -0.01, 0.0), (3.0, -0.01, 0.0), (3.0, 0.01, 0.0), (0.0, 0.01, 0.0), (0.0, -0.01, 0.0)]
window_solid = create_ifcextrudedareasolid(ifcfile, point_list_window_extrusion_area, window_extrusion_placement, (0.0, 0.0, 1.0), window_height)
window_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [window_solid])
window_shape = ifcfile.createIfcProductDefinitionShape(None, None, [window_representation])
window = ifcfile.createIfcWindow(create_guid(), owner_history, "Window", "An awesome window", None, window_placement, window_shape, None, None)

# Relate the window to the opening element
ifcfile.createIfcRelFillsElement(create_guid(), owner_history, None, None, opening_element, window)
ifcfile.write(filename)

In [ ]:
# Create and associate an opening for the door in the wall
opening_placement = create_ifclocalplacement(ifcfile, (window_distance, 0.0, height_from_ground), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), wall_placement)
opening_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
#point_list_opening_extrusion_area = [(0.0, -0.1, 0.0), (3.0, -0.1, 0.0), (3.0, 0.1, 0.0), (0.0, 0.1, 0.0), (0.0, -0.1, 0.0)]
opening_solid = create_ifcextrudedareasolid(ifcfile, point_list_opening_extrusion_area, opening_extrusion_placement, (0.0, 0.0, 1.0), window_height)
opening_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [opening_solid])
opening_shape = ifcfile.createIfcProductDefinitionShape(None, None, [opening_representation])
opening_element = ifcfile.createIfcOpeningElement(create_guid(), owner_history, "Opening", "An awesome opening", None, opening_placement, opening_shape, None)
ifcfile.createIfcRelVoidsElement(create_guid(), owner_history, None, None, wall, opening_element)

# Create a simplified representation for the Door
door_placement = create_ifclocalplacement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0), opening_placement)
door_extrusion_placement = create_ifcaxis2placement(ifcfile, (0.0, 0.0, 0.0), (0.0, 0.0, 1.0), (1.0, 0.0, 0.0))
#point_list_door_extrusion_area = [(0.0, -0.01, 0.0), (3.0, -0.01, 0.0), (3.0, 0.01, 0.0), (0.0, 0.01, 0.0), (0.0, -0.01, 0.0)]
window_solid = create_ifcextrudedareasolid(ifcfile, point_list_window_extrusion_area, window_extrusion_placement, (0.0, 0.0, 1.0), window_height)
window_representation = ifcfile.createIfcShapeRepresentation(context, "Body", "SweptSolid", [window_solid])
window_shape = ifcfile.createIfcProductDefinitionShape(None, None, [window_representation])
window = ifcfile.createIfcDoor(create_guid(), owner_history, "Window", "An awesome window", None, window_placement, window_shape, None, None)

# Relate the window to the opening element
ifcfile.createIfcRelFillsElement(create_guid(), owner_history, 'A common door', 'Description of a standard door', opening_element, window)

In [77]:
doors = ifcfile.by_type("IfcDoor")
doors

[#124=IfcDoor('0LV8Pid0X3IA3jJLVDPidY',#2,'A common door','Description of a standard door',$,#125,#130,$,2.100,0.7)]

In [ ]:
##Height of the Window
# walls.Representation.Representations[1].Items[0][3]
windows = ifcfile.by_type("IfcWindow")
answers = ["yes", "no"]
def get_height_limit():
    tts (f"What is the height limit\n")
    text = []
    text = get_voice()
    element_limit = int(re.findall('\d*\.?\d+', text)[0])

    #short_wall_index = []
    for i in range(0, len(windows)):
        wall_height = windows[i].Representation.Representations[1].Items[0][3]
        if wall_height < element_limit:
            short_wall_index.append(i)
    tts (f"You have {len(walls)} walls and {len(short_wall_index)} of them are shorter than {element_limit} meter.") 
    tts (f"Would you like to hear the ids\n")
    raw_text = get_voice()
    if ([x for x in answers if x in raw_text][0]) == 'yes':
        for i in short_wall_index:
            print(i, walls[i])
        tts (f"The IDs are {short_wall_index} ")

    elif ([x for x in answers if x in raw_text][0]) == 'no':

        tts (f"ok ")

In [ ]:
#Refer a wall in IFC File
def refer_a_wall():
    walls = ifcfile.by_type('IfcWall')
    print(*enumerate(walls), sep='\n')
refer_a_wall()
tts (f"Please select the wall to refer from list\n")
text = []
text = get_voice()
wall_number = int(re.findall('\d*\.?\d+', text)[0])
wall = ifcfile.by_type('IfcWall')[wall_number]
tts (f"You have choosen {wall_number}\n")


In [ ]:
##Cordinates of a wall
x = walls[wall_number].Representation.Representations[1].Items[0].SweptArea.OuterCurve.Points
wall_origin_x = x[0][0][0]
wall_origin_y = x[0][0][1]
x_index1 = x[1][0][0]
x_index2 = x[0][0][0]
y_index1 = x[2][0][1]
y_index2 = x[1][0][1]

height_of_wall = walls[wall_number].Representation.Representations[1].Items[0][3]

if x_index2 - x_index1 > 0:
    wall_x_length = round(x_index2 - x_index1, 3)
else :
    wall_x_length = round((-1)*(x_index2 - x_index1), 3)

if y_index2 - y_index1 > 0:
    wall_y_length = round(y_index2 - y_index1, 3)
else :
    wall_y_length = round((-1)*(y_index2 - y_index1), 3)

if wall_x_length > wall_y_length :
    length_of_wall = wall_x_length
    width_of_wall = wall_y_length
elif wall_x_length < wall_y_length :
    length_of_wall = wall_y_length
    width_of_wall = wall_x_length
wall_y_length



In [ ]:
##Height of the Wall
# walls.Representation.Representations[1].Items[0][3]
answers = ["yes", "no"]
def get_height_limit():
    tts (f"What is the height limit\n")
    text = []
    text = get_voice()
    element_limit = int(re.findall('\d*\.?\d+', text)[0])

    short_wall_index = []
    for i in range(0, len(walls)):
        wall_height = walls[i].Representation.Representations[1].Items[0][3]
        if wall_height < element_limit:
            short_wall_index.append(i)
    tts (f"You have {len(walls)} walls and {len(short_wall_index)} of them are shorter than {element_limit} meter.") 
    tts (f"Would you like to hear the ids\n")
    raw_text = get_voice()
    if ([x for x in answers if x in raw_text][0]) == 'yes':
        for i in short_wall_index:
            print(i, walls[i])
        tts (f"The IDs are {short_wall_index} ")

    elif ([x for x in answers if x in raw_text][0]) == 'no':

        tts (f"ok ")
get_height_limit()

